In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from collections import defaultdict, deque
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns
from scipy.spatial.distance import cdist
from sklearn.metrics.pairwise import cosine_similarity
import torch
import torchvision.transforms as transforms
from PIL import Image
import warnings
warnings.filterwarnings('ignore')


In [2]:
!pip install ultralytics opencv-python deep-sort-realtime numpy

In [3]:
!pip install ultralytics opencv-python deep-sort-realtime numpy

In [4]:
!pip install --upgrade deep-sort-realtime

In [2]:
import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import time
import uuid

# Paths
MODEL_PATH = "/best (2).pt"
VIDEO_PATH = "/15sec_input_720p.mp4"
OUTPUT_PATH = "output_tracked_video_deepsort.mp4"

# Initialize YOLO model
try:
    model = YOLO(MODEL_PATH)
except Exception as e:
    print(f"Error loading YOLO model: {e}")
    exit()

# Initialize DeepSort tracker
try:
    deepsort = DeepSort(max_age=30, nn_budget=100, embedder="mobilenet")
except Exception as e:
    print(f"Error initializing DeepSort: {e}")
    exit()

# Initialize video capture
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps, (width, height))

# Parameters
initial_seconds = 3
initial_frames = int(fps * initial_seconds)
frame_count = 0
player_ids = {}

# Process video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    start_time = time.time()

    # YOLO detection
    try:
        results = model(frame)  # Removed classes filter to detect all classes [ADDED LINE]
    except Exception as e:
        print(f"Error during YOLO inference: {e}")
        continue

    detections = []
    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = box.conf[0]
        cls = int(box.cls[0]) if box.cls.numel() > 0 else -1  # Get class ID [ADDED LINE]
        if conf > 0.3:  # Lowered confidence threshold to 0.3 [ADDED LINE]
            detections.append(([x1, y1, x2 - x1, y2 - y1], conf, cls))
            print(f"Detected class {cls} with confidence {conf:.2f} at frame {frame_count}")  # Debug print [ADDED LINE]

    # Update DeepSort tracker
    try:
        tracks = deepsort.update_tracks(detections, frame=frame)
    except Exception as e:
        print(f"Error updating DeepSort tracks: {e}")
        continue

    # Draw bounding boxes and IDs
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)

        # Store initial IDs
        if frame_count < initial_frames:
            if track_id not in player_ids:
                player_ids[track_id] = {"bbox": [x1, y1, x2, y2], "last_frame": frame_count}

        # Draw bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)

        # Draw ID with background
        text = f"ID: {track_id}"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 2.0
        font_thickness = 2
        text_color = (255, 255, 255)
        text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]
        text_x = x1
        text_y = y1 - 10 if y1 - 10 > 20 else y1 + text_size[1] + 10
        bg_x1, bg_y1 = text_x - 5, text_y - text_size[1] - 5
        bg_x2, bg_y2 = text_x + text_size[0] + 5, text_y + 5
        cv2.rectangle(frame, (bg_x1, bg_y1), (bg_x2, bg_y2), (0, 0, 0), -1)
        cv2.putText(
            frame,
            text,
            (text_x, text_y),
            font,
            font_scale,
            text_color,
            font_thickness,
        )

    # Write frame
    out.write(frame)
    frame_count += 1

    # Simulate real-time
    elapsed_time = time.time() - start_time
    if elapsed_time < 1.0 / fps:
        time.sleep(1.0 / fps - elapsed_time)

    print(f"Processed frame {frame_count}/{total_frames}")

# Release resources
cap.release()
out.release()
print(f"Output video saved to {OUTPUT_PATH}")

Streaming output truncated to the last 5000 lines.
Detected class 2 with confidence 0.92 at frame 94
Detected class 2 with confidence 0.91 at frame 94
Detected class 2 with confidence 0.90 at frame 94
Detected class 2 with confidence 0.90 at frame 94
Detected class 2 with confidence 0.89 at frame 94
Detected class 2 with confidence 0.88 at frame 94
Detected class 2 with confidence 0.88 at frame 94
Detected class 3 with confidence 0.88 at frame 94
Detected class 2 with confidence 0.88 at frame 94
Detected class 2 with confidence 0.87 at frame 94
Detected class 2 with confidence 0.87 at frame 94
Detected class 2 with confidence 0.86 at frame 94
Detected class 2 with confidence 0.86 at frame 94
Detected class 3 with confidence 0.82 at frame 94
Detected class 2 with confidence 0.81 at frame 94
Detected class 0 with confidence 0.76 at frame 94
Detected class 2 with confidence 0.56 at frame 94
Detected class 2 with confidence 0.51 at frame 94
Detected class 2 with confidence 0.50 at frame 94